Import dependencies

In [2]:
import numpy as np
import tensorflow as tf
import pickle
import time as tm

In [2]:
learning_rate = 0.005
training_iteration = 20
batch_size = 64
test_batch_size = 64
display_step = 1

In [3]:
def scale_img(image):
    return ((image/255)-0.5)

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='latin1')
    return dict

def unpickle_all(path):
    train_images = np.zeros((20000,32,32,3))
    test_images = np.zeros((10000,32,32,3))
    train_labels = np.zeros((20000))
    test_labels = np.zeros((10000))
    for i in range(1,3):
        j = i-1
        cifar10 = unpickle(path+'/data_batch_{}'.format(i))
        train_labels[j*10000:(j+1)*10000] = cifar10['labels']
        data = scale_img(cifar10['data'])
        train_images[j*10000:(j+1)*10000,:,:,:] =np.swapaxes(np.reshape(data,(10000,32,32,3), order='F'),axis1=1,axis2=2)

    cifar10 = unpickle(path + '/test_batch')
    test_labels[:] = cifar10['labels']
    data = scale_img(cifar10['data'])
    test_images = np.swapaxes(np.reshape(data, (10000, 32, 32, 3), order='F'),axis1=1,axis2=2)

    return train_images, train_labels, test_images, test_labels

In [14]:
images, labels, test_images, test_labels = unpickle_all('./storage/CIFAR-10')

In [3]:
x = tf.placeholder('float32', [None,32,32,3],name = 'X')
y = tf.placeholder('int64', [None,], name = "Y")
W_CL_1 = tf.Variable(np.random.randn(3, 3, 3, 128), dtype="float32")
W_CL_2 = tf.Variable(np.random.randn(3, 3, 128, 256),dtype="float32")

with tf.name_scope("CL1") as scope:
    CL_1 = tf.nn.conv2d(x,W_CL_1,strides = [1,1,1,1], padding="SAME")
    RELU_1 = tf.nn.relu(CL_1)    
    #MAXPOOL_1 = tf.nn.max_pool(RELU_1,ksize=(1,2,2,1),strides=(1,2,2,1),padding="VALID")
    
with tf.name_scope("CL2") as scope:
    CL_2 = tf.nn.conv2d(RELU_1,W_CL_2,strides = [1,1,1,1], padding="SAME")
    RELU_2 = tf.nn.relu(CL_2)
    MAXPOOL_2 = tf.nn.max_pool(RELU_2,ksize=(1,2,2,1),strides=(1,2,2,1),padding="VALID")
    DROPOUT_2 = tf.nn.dropout(MAXPOOL_2,0.25)
    FLAT_2 = tf.layers.flatten(DROPOUT_2)

with tf.name_scope("DENSE_1") as scope:
    DENSE_3 = tf.contrib.layers.fully_connected(FLAT_2,2048)
    #DROPOUT_3 = tf.nn.dropout(DENSE_3,0.5)

with tf.name_scope("PREDICTIONS") as scope:
    LOGITS_4 = tf.contrib.layers.fully_connected(DENSE_3,10, activation_fn = None)
    y_hat = tf.nn.softmax(LOGITS_4)
    predictions = tf.argmax(y_hat, 1)

with tf.name_scope("ACCURACY") as scope:
    compare = tf.equal(predictions, y)
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(compare, "float"))
    tf.summary.scalar("Accuracy", accuracy)

with tf.name_scope("LOSS") as scope:
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.one_hot(y, depth=10),logits=LOGITS_4))
    tf.summary.scalar("cost_function", loss)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
init = tf.global_variables_initializer()

merged_summary_op = tf.summary.merge_all()

NameError: name 'learning_rate' is not defined

In [21]:
with tf.Session() as sess:
    np.random.seed(1)
    sess.run(init)
    summary_writer = tf.summary.FileWriter('data/logs', graph_def=sess.graph_def)
    # Training cycle
    for iteration in range(training_iteration):
        avg_cost = 0.
        total_batch = int(images.shape[0] / batch_size)
        # Loop over all batches
        total_batch_time = 0.

        for i in range(total_batch):
            start_batch = tm.time()
            batch_xs = images[i*batch_size:(i+1)*batch_size,:,:,:]
            batch_ys = labels[i*batch_size:(i+1)*batch_size]
            _, cost, summary_str = sess.run([optimizer, loss, merged_summary_op], feed_dict={x: batch_xs, y: batch_ys})
            avg_cost += cost/batch_size/total_batch
            summary_writer.add_summary(summary_str, iteration * total_batch + i)
            '''
            stop_batch = tm.time()
            batch_time = stop_batch - start_batch
            total_batch_time += batch_time
            avg_batch_time = total_batch_time / (i+1)
            print("Batch #{}, Batch time: {}, Avg batch time: {}".format(i+1, batch_time, avg_batch_time))
            '''
        if iteration % display_step == 0:
            test_accuracy = sess.run([accuracy], feed_dict={x: test_images, y: test_labels})
            print("Iteration: {}  Cost= {}  Train accuracy= {}%".format(iteration, avg_cost, test_accuracy[0]*100))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\quentin\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-21-9248dd57ae90>", line 16, in <module>
    _, cost, summary_str = sess.run([optimizer, loss, merged_summary_op], feed_dict={x: batch_xs, y: batch_ys})
  File "c:\users\quentin\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\client\session.py", line 877, in run
    run_metadata_ptr)
  File "c:\users\quentin\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\client\session.py", line 1100, in _run
    feed_dict_tensor, options, run_metadata)
  File "c:\users\quentin\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\client\session.py", line 1272, in _do_run
    run_metadata)
  File "c:\users\quentin\appdata\local\programs\python\python36\lib\site-packages\tensorflow\

KeyboardInterrupt: 